In [2]:
pip install bm25s[full] PyStemmer jax[cpu] PySastrawi

In [11]:
import json
import nltk
nltk.download('punkt_tab')

def json_parser(file):
    articles = []
    sent_text = []
    with open(file) as file:
        data = json.load(file)

    counter = 0
    for article in data["articles"]:
        article = article["content"]
        #sent_text += nltk.sent_tokenize(article)

        articles.append(article)
    return articles

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [7]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/dataset_riset/

!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/dataset_riset
indonli_train.jsonl  normalizedcorpus.json


In [12]:
corpus = json_parser("normalizedcorpus.json")
print(corpus[:1])

['Persetujuan hampir tercapai. Jakarta, 13 November: Lord Killearn sebagai ketua sidang lengkap pada pukul 22.00 kemarin malam di Linggardjati, Cirebon, dengan gembira memberikan ucapan selamat kepada kedua belah pihak karena telah tercapai kemungkinan untuk dasar persetujuan. Setelah masing-masing kepala dari kedua delegasi menyampaikan ucapan terima kasih kepada ketua atas usaha beliau dalam perundingan, Prof. Schermerhorn memberitahukan kepada Konsulat bahwa Komisi Jenderal bermaksud berangkat ke Belanda secepat mungkin. Kedua delegasi meminta kepada Lord Killearn agar bersedia memimpin perundingan terakhir jika persetujuan yang pasti telah tercapai. Delegasi Indonesia dan delegasi Belanda hadir lengkap, kecuali Dr. Van Mook. Perundingan yang berjalan lancar selesai pada pukul 23.30 dan berhasil mengumumkan sebuah komite bersama sebagai berikut: Komisi Jenderal dan delegasi Indonesia telah menemukan kemungkinan dasar persetujuan mereka, tetapi persetujuan ini didasarkan pada kesepak

In [15]:
# Tokenize the corpus and index it
corpus_tokens = bm25s.tokenize(corpus)

# Create the BM25 model and index the corpus
retriever = bm25s.BM25(corpus=corpus)
retriever.index(corpus_tokens)

# You can now search the corpus with a query
query = "Pada 12 November 1946, perundingan di Linggardjati, Cirebon, mencapai kesepakatan awal antara delegasi Indonesia dan Komisi Jenderal Belanda di bawah pimpinan Lord Killearn."
query_tokens = bm25s.tokenize(query)

# Get top-k results as a tuple of (doc ids, scores). Both are arrays of shape (n_queries, k).
# To return docs instead of IDs, set the `corpus=corpus` parameter.
results, scores = retriever.retrieve(query_tokens, k=5)

for i in range(results.shape[1]):
    doc, score = results[0, i], scores[0, i]
    print(f"Rank {i+1} (score: {score:.2f}): {doc}")

Split strings:   0%|          | 0/50 [00:00<?, ?it/s]

DEBUG:bm25s:Building index from IDs objects


BM25S Count Tokens:   0%|          | 0/50 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/50 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Rank 1 (score: 14.97): Cirebon, 14 November (Antara): Perundingan di Linggardjati dilaporkan bahwa pada 13 November malam, mulai pukul 22.00, diadakan rapat pleno yang dipimpin oleh Lord Killearn antara delegasi Indonesia dan Komisi Jenderal Belanda. Delegasi Indonesia dan Belanda hadir lengkap, kecuali Dr. Van Mook. Perundingan yang berlangsung lancar selesai pada pukul 23.30 dan menghasilkan sebuah komite bersama. Hasil perundingan ini akan dibawa ke Belanda oleh Komisi Jenderal, yang direncanakan berangkat pada Sabtu, 16 November.

Pada Jumat, 15 November, akan diadakan pertemuan puncak antara delegasi Indonesia dan Belanda untuk meresmikan kesepakatan yang akan dibawa ke Belanda. Diperkirakan Komisi Jenderal akan tiba di Belanda pada Selasa, 19 November, untuk melaporkan hasil perundingan kepada Pemerintah Belanda.

Delegasi Indonesia dijadwalkan berangkat ke Jakarta pada 14 November malam pukul 11.00. Sementara itu, Komisi Jenderal tetap berada di Linggardjati pada 13 November unt